### Notebook to test model for final performance metrics

In [3]:
import sys
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
sys.path.append(os.path.dirname(os.path.abspath('.')))

from dev.utils import load_wandb_model

model, cfg, train_loader, val_loader, test_loader, topo_data, metadata = load_wandb_model('tatlymbz', name='last.ckpt', device='cuda')

/mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/envs/ml-gnn/lib/python3.10/site-packages/tsl/datasets/metr_la.py:98: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(df.index[0], df.index[-1], freq='5T')
/mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/envs/ml-gnn/lib/python3.10/site-packages/tsl/datasets/metr_la.py:109: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  df = df.replace(to_replace=0., method='ffill')


Predicted raw dataset size: 1.36 GB
Predicted raw dataset size: 1.36 GB

Dataloaders and Topology Data Overview:
Batch size: 16
----------------------------------------------------------
Name                 | Samples      | Batches     
----------------------------------------------------------
Train Loader         | 23973        | 1498        
Val Loader           | 3424         | 214         
Test Loader          | 6849         | 428         

Topology Data:
Number of nodes: 207
Number of edges: 1515
----------------------------------------------------------
Imported model from run "tatlymbz".
  | Name                | Type              | Params
----------------------------------------------------------
0 | criterion           | L1Loss            | 0     
1 | input_encoder_patch | Linear            | 256   
2 | input_encoder_node  | Linear            | 128   
3 | edge_encoder        | Linear            | 256   
4 | gnns                | ModuleList        | 99.6 K
5 | U              

In [4]:
# Run test with Lightning trainer
from lightning import Trainer

trainer = Trainer(
    accelerator='auto',
    devices=1,
    logger=None,
    enable_checkpointing=False
)
torch.set_float32_matmul_precision('medium')

# Run test and get metrics
test_results = trainer.test(model, test_loader)


/mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/envs/ml-gnn/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/en ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 428/428 [00:16<00:00, 25.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/loss           0.28419697284698486
       test/mae-12           4.224603176116943
       test/mae-3            3.397914409637451
       test/mae-6            3.704998254776001
      test/mae-all           3.753631830215454
      test/mape-12           10.83359432220459
       test/mape-3           8.767206192016602
       test/mape-6           9.511731147766113
      test/mape-all          9.63846206665039
    test/missing_rate      0.059587497264146805
      test/rmse-12           7.693080902099609
       test/rmse-3           5.990920543670654
       test/rmse-6           6.700266361236572
      test/rmse-all          6